In [1]:
import pandas as pd
import numpy as np
import random
from scipy import stats

In [2]:
PHENO_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PHENO/all-pheno.csv"
PRS_PATH = "/u/project/pasaniuc/kangchen/tmp/prs-1219/REAL-PRS/bmi.tsv.gz"
all_pheno = pd.read_csv(PHENO_PATH)
bmi_prs = pd.read_csv(PRS_PATH, sep="\t")

In [3]:
x_col = 'bmi'
y_col = 'MEAN'
stratify_col = 'AGE'
n_level = 5
n_sample = 10
sample_prop = 0.8

In [4]:
bmi_prs.indiv = bmi_prs.indiv.str.split("_", expand=True)
bmi_prs.indiv = bmi_prs.indiv.astype(int)
bmi_prs = bmi_prs.set_index('indiv')
all_pheno = all_pheno.set_index('ID')
post_all_pheno = all_pheno.reindex(bmi_prs.index)

In [5]:
post_all_pheno = post_all_pheno.dropna(subset=[x_col])
bmi_prs = bmi_prs.dropna(subset=[y_col])

In [8]:
bmi_prs = bmi_prs.loc[post_all_pheno.index, :]

In [10]:
level_li = [i*(1/n_level) for i in range(n_level+1)]
qtl_li = post_all_pheno[stratify_col].quantile(level_li).values

In [11]:
pd.qcut(post_all_pheno['SEX'], q=1)
# qtl_li

indiv
4305423    (-0.001, 1.0]
4360126    (-0.001, 1.0]
4187429    (-0.001, 1.0]
1004389    (-0.001, 1.0]
1387182    (-0.001, 1.0]
               ...      
1498323    (-0.001, 1.0]
4386997    (-0.001, 1.0]
4647541    (-0.001, 1.0]
2937928    (-0.001, 1.0]
2803762    (-0.001, 1.0]
Name: SEX, Length: 478552, dtype: category
Categories (1, interval[float64]): [(-0.001, 1.0]]

In [12]:
grp_index_li = []
for grp_i in range(n_level):
    grp_index_li.append(post_all_pheno.index[(qtl_li[grp_i] <= post_all_pheno[stratify_col]) & (post_all_pheno[stratify_col] <= qtl_li[grp_i+1])])

# grp_index_li.append(post_all_pheno.index[(qtl_li[n_level-1] <= post_all_pheno[stratify_col]) & (post_all_pheno[stratify_col] <= qtl_li[n_level])])

In [13]:
grp_index_li

[Int64Index([4305423, 4360126, 4086323, 4553199, 4677469, 4922412, 1753293,
             5073644, 4460075, 3724943,
             ...
             2120774, 4754456, 4819220, 1966410, 1130254, 3587845, 5375742,
             3725340, 3769165, 3988244],
            dtype='int64', name='indiv', length=98779),
 Int64Index([4187429, 4964808, 4633435, 2675547, 5730058, 4630051, 5383952,
             4677469, 2702849, 2581477,
             ...
             2444435, 3774646, 1118302, 3269435, 1599547, 1651855, 4339931,
             3988244, 5353238, 2937928],
            dtype='int64', name='indiv', length=115410),
 Int64Index([1387182, 4630051, 2908372, 5383952, 5118885, 3552392, 5730405,
             4435628, 5194165, 5847133,
             ...
             2165278, 1350830, 1672502, 5118358, 3269187, 5461955, 4070917,
             5353238, 1498323, 4647541],
            dtype='int64', name='indiv', length=109927),
 Int64Index([3555471, 3364813, 5118885, 4952375, 3031885, 5730405, 1149174,
    

In [14]:
for grp_i in range(n_level):
    post_all_pheno.loc[grp_index_li[grp_i], 'level'] = int(grp_i+1)

In [15]:
post_all_pheno

,SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,...,PC39,PC40,height,SIRE,bmi,cholesterol,hdl_cholesterol,ldl_direct,dilution_factor,level
indiv,,,,,,,,,,,,,,,,,,,,,
4305423,1.0,41.0,-12.17250,5.39163,-1.281030,0.841765,-5.26521,-1.786570,3.109920,-2.630850,...,2.245290,-3.550470,177.0,1001.0,28.2,4.998,1.115,3.463,0.999128,1.0
4360126,0.0,46.0,-13.02450,6.41514,-0.183365,2.927610,-5.88964,0.940534,1.141060,-1.982130,...,2.310350,-1.561490,166.0,1001.0,32.3,7.935,NaN,5.593,0.983840,1.0
4187429,0.0,52.0,-11.47120,3.48383,-1.154580,3.083830,7.65160,-0.913399,-1.548790,1.478930,...,-1.933900,4.429550,158.0,1001.0,20.4,4.906,1.594,2.815,1.005410,2.0
1004389,0.0,65.0,-12.13270,4.02976,-0.988080,0.750294,-2.36431,0.431658,-0.534071,-0.654367,...,0.157072,-1.103580,158.0,1001.0,27.1,7.146,1.742,4.733,1.004870,5.0
1387182,0.0,56.0,-12.21710,3.50821,-1.625990,-1.226800,-5.34580,3.816790,0.579155,-1.169580,...,-0.207469,7.634680,165.5,1001.0,36.3,6.999,1.610,4.495,0.999937,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498323,0.0,56.0,-10.13780,3.40379,1.465470,-4.815010,-3.59211,1.318030,-0.778630,-1.225560,...,-0.154977,1.384810,164.0,1001.0,35.6,5.205,1.011,3.621,0.996949,3.0
4386997,0.0,62.0,-9.21323,2.04166,-2.013420,0.472985,-1.67715,1.026240,0.438598,0.328357,...,-2.005760,-1.826390,155.0,1001.0,21.5,5.529,1.311,3.647,0.996919,4.0
4647541,1.0,60.0,-9.33563,5.06051,0.665526,-0.999905,-5.45859,-0.750495,3.288820,0.131781,...,-4.215390,0.306527,174.0,1001.0,28.8,6.871,1.542,4.519,0.997199,4.0


In [16]:
grp_r2_samples = []
for i_sample in range(n_sample):
    grp_index_random = []
    grp_r2_li = []
    for i in range(n_level):
        grp_index_random.append(random.choices(grp_index_li[i], k=int(0.8*grp_index_li[i].shape[0])))
        res = stats.linregress(post_all_pheno.loc[grp_index_random[i], x_col], bmi_prs.loc[grp_index_random[i], y_col])
        grp_r2_li.append(res.rvalue**2)
    grp_r2_samples.append(grp_r2_li)

In [17]:
avg_grp_r2 = np.mean(grp_r2_samples, axis=0)
std_grp_r2 = np.std(grp_r2_samples, axis=0)
d = {'level': range(1, n_level+1), 'R2': avg_grp_r2, 'R2_std': std_grp_r2}
output = pd.DataFrame(data=d)

In [18]:
output

,level,R2,R2_std
0,1,0.233112,0.001751
1,2,0.239327,0.002146
2,3,0.237317,0.002532
3,4,0.230243,0.002982
4,5,0.221937,0.002263
